In [ ]:
import jtrace
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(jtrace.datadir, "hsc", "HSC.yaml")
telescope = jtrace.Telescope.makeFromYAML(HSC_fn)

In [ ]:
for name, surface in telescope.surfaces.items():
    print(name, surface['surface'])

In [ ]:
plt.figure(figsize=(12, 11))
for surface in telescope.surfaces.values():
    x = np.linspace(-surface['outer'], -surface['inner'], 100, endpoint=True)
    plt.plot(x, [surface['surface'](x_,0) for x_ in x], c='k')
    x = np.linspace(surface['inner'], surface['outer'], 100, endpoint=True)
    plt.plot(x, [surface['surface'](x_,0) for x_ in x], c='k')

p0_left = jtrace.Vec3(-3.5, 0, 0)
p0_right = jtrace.Vec3(3.5, 0, 0)
p0_lmid = jtrace.Vec3(-2.0, 0, 0)
p0_rmid = jtrace.Vec3(2.0, 0, 0)
v_left = jtrace.Vec3(-0.008, 0, -1).UnitVec3()
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()
v_right = jtrace.Vec3(0.008, 0, -1).UnitVec3()

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    for p0 in [p0_left, p0_lmid, p0_rmid, p0_right]:
        ray = jtrace.Ray(p0, v, t=0, w=0.75)
        p0 = ray(-18)
        ray = jtrace.Ray(p0, v, t=0, w=0.75)
        telescope.trace(ray, ax=plt.gca(), color=c)

In [ ]:
plt.figure(figsize=(12, 11))
for surface in telescope.surfaces.values():
    x = np.linspace(-surface['outer'], -surface['inner'], 100, endpoint=True)
    plt.plot(x, [surface['surface'](x_,0) for x_ in x], c='k')
    x = np.linspace(surface['inner'], surface['outer'], 100, endpoint=True)
    plt.plot(x, [surface['surface'](x_,0) for x_ in x], c='k')

p0_left = jtrace.Vec3(-3.5, 0, 0)
p0_right = jtrace.Vec3(3.5, 0, 0)
p0_lmid = jtrace.Vec3(-2.0, 0, 0)
p0_rmid = jtrace.Vec3(2.0, 0, 0)
v_left = jtrace.Vec3(-0.012, 0, -1).UnitVec3()
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()
v_right = jtrace.Vec3(0.012, 0, -1).UnitVec3()
plt.ylim(13.0, 15.5)
plt.xlim(-0.5, 0.5)

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    for p0 in [p0_left, p0_lmid, p0_rmid, p0_right]:
        ray = jtrace.Ray(p0, v, t=0, w=0.75)
        p0 = ray(-18)
        ray = jtrace.Ray(p0, v, t=0, w=0.75)
        telescope.trace(ray, ax=plt.gca(), color=c)

In [ ]:
0.75*np.pi/180